In [4]:
#importing libraries 
import warnings 
warnings.filterwarnings("ignore")
import numpy as np #mathematical operations
import pandas as pd #for reading and manipulating datasets
import nltk 
import spacy
#libraries for NLP tasks
import matplotlib.pyplot as plt #for visualizations
import seaborn as sn #for visualization
%matplotlib inline 
#other libraries to be imported as and when required 

In [2]:
df=pd.read_csv("train_set_taskA - train_set_Fire2022.csv")
display(df.head(10))

,anger,disgust,fear,sadness,surprise,happiness,neutral,Sentences
0,0,0,0,1,1,0,0,محبت کے پردے میں نفرت کرنےوالو
1,0,0,0,0,0,1,0,حامد میر کی خوشی کے لئے اس کا ذمہ دار حکومت ہو...
2,0,0,0,0,0,0,1,اس نئے سال کی شام، ایک سپاہی قاتل (لارنس Fishb...
3,0,0,0,0,0,0,1,کچھ اس طرح سے ایک فلم کے لئے جا سکتے ہیں لیکن ...
4,0,0,0,0,0,0,1,"رون ہاورڈ اور اس کے ""ایڈیٹرز"" صرف اتنا کرنا ....."
5,0,0,0,0,0,0,0,نتائج دنیا کو حیرت میں ڈال سکتی ھے نتائج دس اق...
6,0,0,0,0,1,0,0,ہم غصے خراب موڈ خوشی اور غمی سب میں چپلی کباب ...
7,0,0,0,1,1,0,0,اب ایسی حیرت و وارفتگی کو کیا کہیے دعا کو ہاتھ...
8,0,0,0,1,1,0,0,دے تو رہے ہیں آپ اپنی آنکھوں سے نفرت کی پٹی ات...
9,0,0,0,0,1,0,0,حیرت ہوئی


In [5]:
#getting tf-idf 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
stopwords=set(stopwords.words("urdu"))
print(stopwords)

{'عظین', 'ضکے', 'ضرورت', 'خیطبکہ', 'کھولٌب', 'اہن', 'کیطے', 'کورے', 'خت', 'ترتیت', 'ہر', 'لازهی', 'اکٹھے', 'ضکب', 'خوًہی', 'ڈھوًڈیں', 'آخر', 'کیوًکہ', 'گٌتی', 'ضوچتے', 'کہب', 'کےثعذ', 'پراًب', 'زبضر', 'هیری', 'ثڑے', 'خبرہب', 'تھوڑا', 'کوًطے', 'علاوٍ', 'ضبتھ', 'کہے', 'لوگوں', 'قجیلہ', 'رریعہ', 'خبهوظ', 'طریقہ', 'چکیں', 'اى', 'کرتی', 'کہتے', 'چبر', 'زیبدٍ', 'اوًچے', 'خططرذ', 'ضوچب', 'کوتر', 'ہوچکے', 'درخہ', 'ًے', 'ثعذ', 'تن', 'ہوًب', 'کرتبہوں', 'فی', 'اش', 'کہوں', 'اکیلے', 'اکٹھب', 'درضت', 'تھے', 'دیتی', 'لیٌب', 'ض', 'ًئے', 'دی', 'تریي', 'ثڑا', 'کوًطب', 'ہورہی', 'ثٌبرہے', 'هسترهہ', 'ب', 'کہی', 'ضوچ', 'ثبلا', 'هسترم', 'لا', 'پیع', 'والے', 'دکھبتے', 'ج', 'دیب', 'ڈھوًڈا', 'درزقیقت', 'رکھتے', 'چھہ', 'خٌبة', 'تھب', 'چھوٹوں', 'ہوتی', 'لو', 'هڑے', 'هلا', 'ظبہر', 'اکیلا', 'کررہی', 'دے', 'ًو', 'طور', 'دکھبًب', 'لگتی', 'غخص', 'ًبپطٌذ', 'رکھب', 'فرد', 'توبم', 'هسطوش', 'گئے', 'ًئی', 'ثبرے', 'ٹھیک', 'ضوچیں', 'صبف', 'اوًچبئی', 'ضت', 'لڑکپي', 'ثٌبرہی', 'دلچطپ', 'ثلٌذ', 'هڑًب', 'گیب', 'کب', 'گروہوں', 'غ

In [9]:
def vector_fit(data):
    #finding the unique word and then creating the vocabulary 
    unique_word=set()
    for each_sentence in data:
        for each_word in each_sentence.split(" "):
            if len(each_word)>3:
                unique_word.add(each_word)
    print(f" Number of unique words in the dataset are {len(unique_word)}")
    #removing stop words from the set of unique_words 
    for i in list(unique_word):
        if i in stopwords:
            unique_word.remove(i)
        else:
            pass
    print(f"Left number of unique words in the see are {len(unique_word)}")
    #Now that we have unique words, we have to sort them and index them 

    vocab={}

    for index,word in enumerate(sorted(list(unique_word))):
        vocab[word]=index
    return vocab,unique_word


In [18]:
def vector_transform(data):
    #getting count values for each vocab
    vocab,_=vector_fit(data)
    row,col,val=[],[],[]# to store sparse matrix
    from collections import Counter
    from scipy.sparse import csr_matrix #to  create sparse matrix


    for idx,sentence in enumerate(list(data)):
        count_word=dict(Counter(sentence.split(" ")))
        for word,count in count_word.items():
            if len(word)<3 and word in stopwords:
                pass
            else:
 #try and except block to avoid errors as matching urdu can be challenging           
                try:
                    col_index=vocab.get(word)#taking index of that word
                    if col_index>=0:
                        row.append(idx)
                        col.append(col_index)
                        val.append(count)
                except TypeError:
                    pass
    print( "The sparse matrix so obtained is : \n",csr_matrix((val,(row,col)),shape=(len(list(data)),len(vocab))).toarray())
    return csr_matrix((val,(row,col)),shape=(len(list(data)),len(vocab))).toarray()


In [19]:
vocab,unique_word=vector_fit(df["Sentences"])

 Number of unique words in the dataset are 43372
Left number of unique words in the see are 43287


In [20]:
sparse_matrix=vector_transform(df["Sentences"])

 Number of unique words in the dataset are 43372
Left number of unique words in the see are 43287
The sparse matrix so obtained is : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
